### Goal: Make IPL interesting for Vipul with the help of the data visualization(facts, trivia, stats, etc)
#### Constraints:
- Vipul doest have much time so graphs/plots should easy to understand
- No additional data other than the provided
 

## Dependency

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import missingno
import os
%matplotlib inline

In [8]:
plt.style.use('seaborn')

In [13]:
# read the data
df_bb = pd.read_csv("../src/data/Ball_by_Ball.csv")
df_match = pd.read_csv("../src/data/Match.csv")
df_player_match = pd.read_csv("../src/data/Player_Match.csv")
df_player = pd.read_csv("../src/data/Player.csv")
df_season = pd.read_csv("../src/data/Season.csv")
df_team = pd.read_csv("../src/data/Team.csv")

### **Missing Value inspection**

In [18]:
df_match.head()

,Match_Id,Match_Date,Team_Name_Id,Opponent_Team_Id,Season_Id,Venue_Name,Toss_Winner_Id,Toss_Decision,IS_Superover,IS_Result,Is_DuckWorthLewis,Win_Type,Won_By,Match_Winner_Id,Man_Of_The_Match_Id,First_Umpire_Id,Second_Umpire_Id,City_Name,Host_Country
0,335987,18-Apr-08,2,1,1,M Chinnaswamy Stadium,2,field,0,1,0,by runs,140.0,1.0,2.0,470,477,Bangalore,India
1,335988,19-Apr-08,4,3,1,"Punjab Cricket Association Stadium, Mohali",3,bat,0,1,0,by runs,33.0,3.0,19.0,471,487,Chandigarh,India
2,335989,19-Apr-08,6,5,1,Feroz Shah Kotla,5,bat,0,1,0,by wickets,9.0,6.0,90.0,472,512,Delhi,India
3,335990,20-Apr-08,7,2,1,Wankhede Stadium,7,bat,0,1,0,by wickets,5.0,2.0,11.0,473,476,Mumbai,India
4,335991,20-Apr-08,1,8,1,Eden Gardens,8,bat,0,1,0,by wickets,5.0,1.0,4.0,474,486,Kolkata,India
